In [1]:
DATASET_TRAIN_FILE = "/content/drive/My Drive/project/dataset/preprocessed/sentiment_analysis/sentiment_analysis_train.csv"
DATASET_TEST_FILE = "/content/drive/My Drive/project/dataset/preprocessed/sentiment_analysis/sentiment_analysis_test.csv"
TWEETS_FILE = "/content/drive/My Drive/project/dataset/preprocessed/tweets/dataset_model.csv"

In [2]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub

In [3]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
train = pd.read_csv(DATASET_TRAIN_FILE).values
test = pd.read_csv(DATASET_TEST_FILE).values

print(train.shape)
print(test.shape)

train_x = embed(train[:, 0]).numpy()
train_y = train[:, 1].astype(np.int)
test_x = embed(test[:, 0]).numpy()
test_y = test[:, 1].astype(np.int)

(71999, 2)
(8000, 2)


In [25]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.naive_bayes import CategoricalNB

In [55]:
n_bins = int((train_x.max() - train_x.min()) / 0.001)

In [56]:
discretizer = KBinsDiscretizer(n_bins=n_bins, encode='ordinal')

In [57]:
train_x_d = discretizer.fit_transform(train_x)

In [39]:
test_x_d = discretizer.transform(test_x)

# Categorical Naive Bayes

In [58]:
model = CategoricalNB()

model.fit(train_x_d[0:55000], train_y[0:55000])

predicted_y = model.predict(test_x_d)
print("Accuracy: ", accuracy_score(predicted_y, test_y))
print("Precision: ", precision_score(predicted_y, test_y))
print("Recall: ", recall_score(predicted_y, test_y))

Accuracy:  0.75575
Precision:  0.7575308641975309
Recall:  0.7594059405940594


In [61]:
tweets_np = pd.read_csv(TWEETS_FILE).values
tweets_np = tweets_np[tweets_np[:, 0] == "realDonaldTrump"]

tweets_rdt = tweets_np[:, 2].tolist()
embeddings_rdt = discretizer.transform(embed(tweets_rdt).numpy())

In [62]:
result = model.predict(embeddings_rdt)

print(np.unique(result, return_counts=True))

(array([0, 1]), array([4559, 7481]))


In [63]:
for i in range(result.shape[0]):
  print(tweets_rdt[i], result[i])

Streaming output truncated to the last 5000 lines.
We are breaking all Jobs and Economic Records but, importantly, our Country has TREMENDOUS FUTURE POTENTIAL. We have just begun! 1
“To this point, President Trump’s achievements are unprecedented.” @LouDobbs 1
Dave Hughes is running for Congress in the Great State of Minnesota. He will help us accomplish our America First policies, is strong on Crime, the Border, our 2nd Amendmen, Trade, Military and Vets. Running against Pelosi Liberal Puppet Petterson. Dave has my Total Endorsement! 1
14 days for $28 MILLION - $2 MILLION a day, No Collusion. A great day for America! 1
“Unprecedented Jobs Growth Streak Continues as Wages Rise” 0
Under our horrible immigration laws, the Government is frequently blocked from deporting criminal aliens with violent felony convictions. House GOP just passed a bill to increase our ability to deport violent felons (Crazy Dems opposed). Need to get this bill to my desk fast! 0
The Woodward book is a scam. I d